In [1]:
import pandas
from os import listdir

files = [pandas.read_csv('out/' + file, sep=';')
         for file in listdir('out') if not file.startswith('scores-')]

df = pandas.concat(files)

In [2]:
trimmed = df[['Algorithm', 'Ranking Factor', 'Size Factor', 'Index', 'Line', 'Character']]
trimmed.loc[trimmed['Ranking Factor'].isnull(), 'Ranking Factor'] = 0
trimmed.loc[trimmed['Size Factor'].isnull(), 'Size Factor'] = 0
groups = trimmed.groupby(['Algorithm', 'Ranking Factor', 'Size Factor'])

/tmp/ipykernel_51863/3325441677.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trimmed.loc[trimmed['Ranking Factor'].isnull(), 'Ranking Factor'] = 0
/tmp/ipykernel_51863/3325441677.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trimmed.loc[trimmed['Size Factor'].isnull(), 'Size Factor'] = 0


In [3]:
from collections import defaultdict
data = []
for groupName, frame in groups:
    line = -1
    ch = -1
    lemmaPosition = -1
    thisdata = defaultdict(lambda: defaultdict(int))
    positionCounts = defaultdict(int)
    for resultIndex, l, c in frame[['Index', 'Line', 'Character']].values:
        if line != l:
            line = l
            ch = c
            lemmaPosition = 0
        if ch != c:
            if ch + 1 == c:
                lemmaPosition += 1
            else:
                lemmaPosition = 0
            ch = c
        thisdata[lemmaPosition][resultIndex] += 1
        positionCounts[lemmaPosition] += 1
        # print(resultIndex, l, c, lemmaPosition)
    grades = defaultdict(int)
    for lemmaPosition, results in thisdata.items():
        attempts = positionCounts[lemmaPosition]
        grades[lemmaPosition] = sum(map(lambda j: results[j] / (j + 1), range(10))) / attempts
    a = sum(map(lambda x: grades[x] / (x + 1), range(len(grades))))
    b = sum(map(lambda x: 1 / (x+1), range(len(grades))))
    score = 100 * ( a / b )
    data.append((score, groupName))
data.sort(reverse=True)
data

[(54.36873151658046, ('StructuredTypeEvaluation', 10, 10))]